# Video search - embedding 작업 시작

Nova Multimodal Embedding (MME)은 통합된 의미 공간 내에서 비디오, 텍스트 및 이미지를 표현하여 지능형 비디오 검색을 가능하게 합니다. 이를 통해 사용자는 자연어 프롬프트 또는 시각적 예제를 사용하여 비디오를 검색하고 높은 정확도로 관련 순간을 검색할 수 있습니다.

![nova-mme](../statics/nova-mme-diagram.png)

Video Understanding Tool UI를 통해 직접 또는 프로그래밍 인터페이스를 통해 비디오를 업로드할 수 있습니다.

이 샘플에서는 비디오 검색을 위한 Nova Multimodal Embedding을 실험합니다. UI를 통해 비디오를 업로드하는 대신 이 노트북을 사용하여 여러 비디오를 업로드하며, 이는 검토를 위해 tool의 UI에서 사용할 수 있습니다.

frame 기반 및 clip 기반 처리와 같은 다른 워크플로우에 대한 비디오 task를 시작하는 방법을 보여주는 샘플 노트북도 동일한 폴더에서 찾을 수 있습니다.

이 샘플 노트북에서는 다음을 수행합니다:
1. 공개 URL에서 비디오 다운로드
2. S3 데이터 bucket에 업로드
3. 각 비디오에 대한 Nova MME embedding task 시작
4. heartbeat 확인으로 task 진행 상황 모니터링
5. tool의 검색 기능을 통한 embedding 검색
6. 정리

In [ ]:
import boto3
import json
import uuid
import requests
import os
import time
from urllib.parse import urlparse
from pathlib import Path

## AWS Client 및 구성 설정

AWS client를 초기화하고 계정 정보를 가져옵니다. Video Understanding Tool은 다음 명명 규칙을 따르는 S3 bucket에 데이터를 저장합니다: `bedrock-mm-{account_id}-{region}`.

In [ ]:
# AWS client 초기화
session = boto3.session.Session()
sts = session.client("sts")
lambda_client = boto3.client("lambda")
s3_client = boto3.client("s3")

# 계정 및 region 정보 가져오기
account_id = sts.get_caller_identity()["Account"]
region = session.region_name

print(f"Account ID: {account_id}")
print(f"Region: {region}")

In [ ]:
# 구성 상수
S3_BUCKET = f"bedrock-mm-{account_id}-{region}"
S3_KEY_TASK_TEMPLATE = "tasks/{task_id}/upload/{file_name}"
LAMBDA_FUN_NAME_NOVA_MME_START_TASK = "bedrock-mm-nova-srv-start-task"
LAMBDA_FUN_NAME_NOVA_MME_GET_TASK = "bedrock-mm-nova-srv-get-video-task"
LAMBDA_FUN_NAME_NOVA_MME_SEARCH_VECTOR = "bedrock-mm-nova-srv-search-vector"
LAMBDA_FUN_NAME_NOVA_MME_DELETE_TASK = "bedrock-mm-nova-srv-delete-video-task"
NOVA_MME_MODEL_ID = "amazon.nova-2-multimodal-embeddings-v1:0"

In [ ]:
# 로컬 다운로드 디렉토리 생성
DOWNLOAD_DIR = "./downloaded_videos"
Path(DOWNLOAD_DIR).mkdir(exist_ok=True)

## 공개 비디오 URL 정의

여기에 공개 비디오 URL을 추가하십시오. 이는 공개적으로 액세스할 수 있는 비디오 파일(mp4, mov, avi 등)에 대한 직접 링크여야 합니다.

In [ ]:
# 처리할 공개 비디오 URL 목록
video_urls = [
    {
        "url": "https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/8082573f-f39e-4e39-a48f-f3562cc6e597/aws-ads-magical-day-out.mp4",
        "name": "aws-ads-magical-day-out.mp4",
        "description": "샘플 비디오 - AWS Ads Magical Day Out"
    },
    {
        "url": "https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/8082573f-f39e-4e39-a48f-f3562cc6e597/aws-ads-rainy-day.mp4",
        "name": "aws-ads-rainy-day.mp4",
        "description": "샘플 비디오 - AWS Ads Rainy Day Out"
    },
    {
        "url": "https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/8082573f-f39e-4e39-a48f-f3562cc6e597/aws-ads-transform-genai.mp4",
        "name": "aws-ads-transform-genai.mp4",
        "description": "샘플 비디오 - AWS Ads Transform GenAI"
    },
    {
        "url": "https://ws-assets-prod-iad-r-iad-ed304a55c2ca1aee.s3.us-east-1.amazonaws.com/8082573f-f39e-4e39-a48f-f3562cc6e597/nova-reel-burger.mp4",
        "name": "nova-reel-burger.mp4",
        "description": "샘플 비디오 - Nova Reel 생성 버거 광고"
    },
    # 필요에 따라 더 많은 비디오 URL 추가
]

print(f"처리할 비디오 {len(video_urls)}개 발견")
for i, video in enumerate(video_urls, 1):
    print(f"{i}. {video['name']} - {video['description']}")

## 공개 URL에서 비디오 다운로드

S3에 업로드하기 전에 공개 URL에서 각 비디오를 로컬 저장소로 다운로드합니다.

In [ ]:
def download_videos(video_list, download_dir="/tmp"):
    """
    URL에서 비디오 목록을 다운로드하고 각 항목에 'local_path' 키를 추가합니다.

    Args:
        video_list (list): 각각 'url'과 'name'을 포함하는 dict 목록.
        download_dir (str): 비디오를 저장할 로컬 디렉토리 (기본값: /tmp).

    Returns:
        list: 각 항목에 'local_path'가 추가된 업데이트된 목록.
    """
    os.makedirs(download_dir, exist_ok=True)
    
    for video in video_list:
        url = video.get("url")
        filename = video.get("name")
        local_path = os.path.join(download_dir, filename)

        print(f"{filename} 다운로드 중...")
        response = requests.get(url, stream=True)
        response.raise_for_status()

        with open(local_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)

        video["local_path"] = local_path
        print(f"{local_path}에 저장됨")

    return video_list

downloaded_videos = download_videos(video_urls, DOWNLOAD_DIR)
print(json.dumps(downloaded_videos, indent=2, ensure_ascii=False))

## S3에 비디오 업로드 및 Embedding Task 시작

다운로드한 각 비디오를 S3에 업로드하고 처리를 위한 Nova MME embedding task를 시작합니다. 각 task는 완료될 때까지 heartbeat 확인으로 모니터링됩니다.

In [ ]:
def get_task_status(task_id):
    """task의 현재 상태 가져오기"""
    try:
        request = {"TaskId": task_id}
        response = lambda_client.invoke(
            FunctionName=LAMBDA_FUN_NAME_NOVA_MME_GET_TASK,
            InvocationType="RequestResponse",
            Payload=json.dumps(request),
        )
        
        response_payload = json.loads(response["Payload"].read())
        if response.get('StatusCode') == 200:
            # 응답 body가 문자열인 경우 파싱
            if isinstance(response_payload.get('body'), str):
                body = json.loads(response_payload['body'])
            else:
                body = response_payload.get('body', response_payload)
            
            return body.get('Status')
        else:
            print(f"❌ task 상태 가져오기 실패: {response_payload}")
            return 'error'
            
    except Exception as e:
        print(f"❌ {task_id}의 task 상태 가져오기 오류: {str(e)}")
        return 'error'

def wait_for_task_completion(task_id, file_name, check_interval=5, max_wait_time=1800):
    """heartbeat 모니터링으로 task 완료 대기"""
    print(f"⏳ {file_name}에 대한 task {task_id} 모니터링 중...")
    start_time = time.time()
    
    while (time.time() - start_time) < max_wait_time:
        status = get_task_status(task_id)
        print(f"  📋 상태: {status}")
        
        if status == 'completed':
            print(f"✅ Task {task_id} 성공적으로 완료!")
            return True
        elif status in ['failed', 'error']:
            print(f"❌ Task {task_id} 실패, 상태: {status}")
            return False
        
        # 다음 확인 전 대기
        time.sleep(check_interval)
    
    print(f"⚠️ Task {task_id}가 {max_wait_time}초 후 시간 초과")
    return False

In [ ]:
def upload_and_start_task(video_info):
    """S3에 비디오 업로드 및 Nova MME embedding task 시작"""
    try:
        # 고유한 task ID 생성
        task_id = str(uuid.uuid4())
        file_name = video_info['name']
        local_path = video_info['local_path']
        
        # S3 key 구성
        s3_key = S3_KEY_TASK_TEMPLATE.format(task_id=task_id, file_name=file_name)
        
        print(f"📤 {file_name}을 S3에 업로드 중...")
        s3_client.upload_file(local_path, S3_BUCKET, s3_key)
        print(f"✓ s3://{S3_BUCKET}/{s3_key}에 업로드됨")
        
        # Nova MME task 요청 준비
        request = {
            "ModelId": NOVA_MME_MODEL_ID,
            "EmbedMode": "AUDIO_VIDEO_COMBINED",
            "DurationS": 5,
            "TaskId": task_id,
            "FileName": file_name,
            "TaskName": video_info.get('description',file_name),
            "Video": {
                "S3Object": {
                    "Bucket": S3_BUCKET,
                    "Key": s3_key
                }
            },
            "RequestBy": "workshop_user"
        }
        
        print(f"🚀 {file_name}에 대한 Nova MME embedding task 시작 중...")
        response = lambda_client.invoke(
            FunctionName=LAMBDA_FUN_NAME_NOVA_MME_START_TASK,
            InvocationType="RequestResponse",
            Payload=json.dumps(request),
        )
        
        response_payload = json.loads(response["Payload"].read())
        if response.get('StatusCode') == 200:
            if "body" not in response_payload:
                print(response_payload)
            print(f"✓ {file_name}에 대한 task {task_id} 성공적으로 시작됨")
            
            # heartbeat 모니터링으로 task 완료 대기
            task_completed = wait_for_task_completion(task_id, file_name)
            
            return {
                'task_id': task_id,
                'file_name': file_name,
                's3_key': s3_key,
                'status': 'completed' if task_completed else 'failed',
                'response': response_payload,
                **video_info
            }
        else:
            print(f"❌ {file_name}에 대한 task 시작 실패: {response_payload}")
            return None
        
    except Exception as e:
        print(f"❌ {video_info['name']} 처리 오류: {str(e)}")
        return None

# 다운로드한 모든 비디오 처리
processing_tasks = []
for video_info in downloaded_videos:
    task_info = upload_and_start_task(video_info)
    if task_info:
        processing_tasks.append(task_info)
    print()

print(f"{len(processing_tasks)}개의 embedding task 처리됨")
successful_tasks = [t for t in processing_tasks if t['status'] == 'completed']
failed_tasks = [t for t in processing_tasks if t['status'] == 'failed']

print(f"✅ 성공: {len(successful_tasks)}")
print(f"❌ 실패: {len(failed_tasks)}")

for task in processing_tasks:
    print(f"  - Task ID: {task['task_id']} | 파일: {task['file_name']} | 상태: {task['status']}")

## Embedding 검색

Video Understanding Tool UI를 사용하여 텍스트 또는 이미지 입력을 사용하여 embedding 검색을 수행할 수도 있습니다. 프로그래밍 방식 액세스의 경우 UI를 구동하는 동일한 API 또는 Lambda 함수를 애플리케이션 통합에 사용할 수 있습니다.

Video Understanding Tool은 직접 쿼리할 수 있는 Amazon S3 vector store에 embedding을 저장합니다. 또는 아래 예제와 같이 UI를 통해 검색할 수 있습니다. UI는 추가 task 관련 메타데이터를 제공하고 해당 비디오를 표시하기 위한 사전 서명된 S3 URL을 생성합니다.

In [ ]:
request = {
    "SearchText": "aws",
    "Source": "",
    "InputType": "text",
    "InputBytes": None,
    "InputFormat": "",
    "RequestBy": "workshop_user",
    "PageSize": 8,
    "FromIndex": 0,
}
response = lambda_client.invoke(
            FunctionName=LAMBDA_FUN_NAME_NOVA_MME_SEARCH_VECTOR,
            InvocationType="RequestResponse",
            Payload=json.dumps(request),
        )
response_payload = json.loads(response["Payload"].read())
print(json.dumps(response_payload,indent=2, ensure_ascii=False))

## 정리 (선택 사항)

task를 삭제하고 다운로드한 파일을 제거하여 리소스를 정리합니다. **경고: 이렇게 하면 embedding을 포함한 모든 task 데이터가 영구적으로 삭제됩니다.**

In [ ]:
def delete_nova_mme_task(task_id):
    request = {"TaskId": task_id}
    response = lambda_client.invoke(
                FunctionName=LAMBDA_FUN_NAME_NOVA_MME_DELETE_TASK,
                InvocationType="RequestResponse",
                Payload=json.dumps(request),
            )
    
    response_payload = json.loads(response["Payload"].read())
        
    if response.get('StatusCode') == 200:
            print(f"✓ task {task_id} 성공적으로 삭제됨")
    else:
        print(f"❌ task {task_id} 삭제 실패: {response_payload}")
        return None

import shutil
import os

def delete_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"폴더 삭제됨: {folder_path}")
    else:
        print(f"폴더를 찾을 수 없음: {folder_path}")

# 비디오 및 task를 삭제하려면 다음 코드의 주석을 해제하십시오.
'''
# 모든 task 삭제
for task in processing_tasks:
     print(f"파일에 대한 task 삭제 중: {task['file_name']}")
     delete_nova_mme_task(task["task_id"])

# 로컬 디스크에서 다운로드한 비디오 삭제
delete_folder(DOWNLOAD_DIR)
'''